In [1]:
import pandas as pd
import numpy as np

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Pablo94/xlm-r-racismo-es-v2-finetuned-detests")
model = AutoModelForSequenceClassification.from_pretrained("Pablo94/xlm-r-racismo-es-v2-finetuned-detests")

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

D:\Anaconda\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in D:\hf_cache\hub\models--Pablo94--xlm-r-racismo-es-v2-finetuned-detests. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/803 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [6]:
bias_corp = pd.read_csv("bias_corp_racial.csv")
df = bias_corp.copy()
# Function to compute the normalized confidence-weighted bias score
def compute_weighted_bias(row):
    # Extract bias and confidence scores
    bias_scores = np.array([row['bias_1'], row['bias_2'], row['bias_3']])
    confidence_scores = np.array([row['confidence_1'], row['confidence_2'], row['confidence_3']])
    
    # Normalize confidence scores so they sum to 1
    normalized_conf = confidence_scores / confidence_scores.sum()
    
    # Compute the weighted bias score
    weighted_bias = np.dot(bias_scores, normalized_conf)
    return weighted_bias

# Apply the function to compute weighted bias scores
df['weighted_bias'] = df.apply(compute_weighted_bias, axis=1)

# Determine true labels based on a threshold (e.g., weighted bias >= 3 is racist)
df['true_label'] = (df['weighted_bias'] >= 3).astype(int)
df.head()

,text,bias_1,bias_2,bias_3,confidence_1,confidence_2,confidence_3,time_1,time_2,time_3,sentiment_score,source,weighted_bias,true_label
0,"'' Democrats needed someone like Obama, half w...",4,3,2,4,3,5,82,94,295,0.205548,BB,2.916667,0
1,'Greatest threat to the Black Famiky is the wh...,1,3,4,6,6,4,48,37,23,-0.189737,BB,2.500000,0
2,"'In March, Black Lives Matter activists marche...",3,3,2,7,7,7,140,2495,75,-0.282843,BB,2.666667,0
3,"'Rogue'' Chinese Virologist Joins Twitter, Pub...",5,4,3,8,9,7,3900,6838,85,-0.094491,BB,4.041667,1
4,'What this means is he could have told black p...,2,3,3,8,7,6,9302,4203,43,-0.347440,BB,2.619048,0


In [8]:
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Function to make predictions
def predict_texts(df):
    predictions = []
    model.eval()
    with torch.no_grad():
        for text in tqdm(df['text'], desc="Predicting..."):
            inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            outputs = model(**inputs)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).tolist()
            predictions.append(preds)
    return predictions

# Predict on df['text']
df['racism_prediction'] = predict_texts(df)

# Display the results
df.head()

Predicting...: 100%|█████████████████████████████████████████████████████████████| 43732/43732 [28:37<00:00, 25.46it/s]


,text,bias_1,bias_2,bias_3,confidence_1,confidence_2,confidence_3,time_1,time_2,time_3,sentiment_score,source,weighted_bias,true_label,racism_prediction
0,"'' Democrats needed someone like Obama, half w...",4,3,2,4,3,5,82,94,295,0.205548,BB,2.916667,0,[0]
1,'Greatest threat to the Black Famiky is the wh...,1,3,4,6,6,4,48,37,23,-0.189737,BB,2.500000,0,[0]
2,"'In March, Black Lives Matter activists marche...",3,3,2,7,7,7,140,2495,75,-0.282843,BB,2.666667,0,[0]
3,"'Rogue'' Chinese Virologist Joins Twitter, Pub...",5,4,3,8,9,7,3900,6838,85,-0.094491,BB,4.041667,1,[0]
4,'What this means is he could have told black p...,2,3,3,8,7,6,9302,4203,43,-0.347440,BB,2.619048,0,[0]


In [10]:
df.to_csv("xlm-r-racismo results.csv",index=False)

### DATASET 2: SAKREN

In [6]:
from datasets import load_dataset

sakren = load_dataset("sakren/twitter_racism_dataset")

In [8]:
sakren = sakren['train'].to_pandas()
sakren.head()

,index,id,Text,Annotation,oh_label
0,5.767493e+17,5.767493e+17,@AAlwuhaib1977 Muslim mob violence against Hin...,racism,1
1,5.408905e+17,5.408905e+17,@Te4m_NiGhtM4Re http://t.co/5Ih7MkDbQG,none,0
2,5.678433e+17,5.678433e+17,@jncatron @isra_jourisra @AMPalestine Islamoph...,racism,1
3,5.766462e+17,5.766462e+17,"Finally I'm all caught up, and that sudden dea...",none,0
4,5.713492e+17,5.713492e+17,@carolinesinders @herecomesfran *hugs*,none,0


In [10]:
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def predict_texts(df):
    predictions = []
    model.eval()
    with torch.no_grad():
        for text in tqdm(df, desc="Predicting..."):
            inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)
            outputs = model(**inputs)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).tolist()
            predictions.append(preds)
    return predictions

sakren['prediction'] = predict_texts(sakren['Text'])

Predicting...: 100%|█████████████████████████████████████████████████████████████| 13471/13471 [08:12<00:00, 27.38it/s]


In [19]:
sakren.head()

,index,id,Text,Annotation,oh_label,prediction
0,5.767493e+17,5.767493e+17,@AAlwuhaib1977 Muslim mob violence against Hin...,racism,1,[1]
1,5.408905e+17,5.408905e+17,@Te4m_NiGhtM4Re http://t.co/5Ih7MkDbQG,none,0,[0]
2,5.678433e+17,5.678433e+17,@jncatron @isra_jourisra @AMPalestine Islamoph...,racism,1,[1]
3,5.766462e+17,5.766462e+17,"Finally I'm all caught up, and that sudden dea...",none,0,[0]
4,5.713492e+17,5.713492e+17,@carolinesinders @herecomesfran *hugs*,none,0,[0]


In [33]:
sakren.to_csv("xlm_sakren.csv",index=False)

### DATASET 3: KAGGLE

In [11]:
kaggle_racism = pd.read_csv("D:/OneDrive - University of Georgia/UGA/Dr. Itai/Racism/racism_kaggle/racism1.csv",low_memory=False)
kaggle = kaggle_racism[['date','tweet','cleanText','TextBlob Sentiment']]
kaggle_racism.head()

,Unnamed: 0,id,conversation_id,created_at,date,time,timezone,user_id,username,name,...,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,cleanText,TextBlob Score,TextBlob Sentiment
0,0,1.400600e+18,1.400210e+18,2021-06-03 18:59:56 Central Daylight Time,6/3/2021,18:59:56,-500,9.292310e+17,rayne_katsune,Rayne ã€ŒThe Nuisanceã€ Katsune ã‚µã‚¿ã‚¢ãƒ«,...,NaN,NaN,"[{'screen_name': 'HyperSnoc', 'name': 'Your Lo...",NaN,NaN,NaN,NaN,hypersnoc echo im_not_kit racism,0.00,Neutral
1,1,1.400600e+18,1.397240e+18,2021-06-03 18:59:55 Central Daylight Time,6/3/2021,18:59:55,-500,5.791119e+08,tonyhasanidea,no one you know,...,NaN,NaN,"[{'screen_name': 'manoutdoors4', 'name': 'Stoi...",NaN,NaN,NaN,NaN,manoutdoor aj_lady_liberti fbiwfo thejusticede...,0.05,Positive
2,2,1.400600e+18,1.400600e+18,2021-06-03 18:59:46 Central Daylight Time,6/3/2021,18:59:46,-500,3.098130e+09,americanist4u,Americanist (Racism and treason are not patrio...,...,NaN,NaN,[],NaN,NaN,NaN,NaN,trumpisnotwel,0.00,Neutral
3,3,1.400600e+18,1.400590e+18,2021-06-03 18:59:44 Central Daylight Time,6/3/2021,18:59:44,-500,1.084560e+18,mimizwei,MimiRocheðŸ›¡,...,NaN,NaN,"[{'screen_name': 'KateWilliamsme', 'name': 'Pr...",NaN,NaN,NaN,NaN,katewilliamsm annabeth ðÿš racism exempt queen...,0.00,Neutral
4,4,1.400600e+18,1.400600e+18,2021-06-03 18:59:42 Central Daylight Time,6/3/2021,18:59:42,-500,3.071625e+09,goseemickeyron,Honored to be a Disney Legend,...,NaN,NaN,"[{'screen_name': 'WGNNews', 'name': 'WGN TV Ne...",NaN,NaN,NaN,NaN,wgnnew critic race theoryâ noth teach hatr big...,0.00,Neutral


In [12]:
kaggle.dropna(inplace=True)
kaggle.head()

C:\Users\MUDIT BAID\AppData\Local\Temp\ipykernel_13200\3045440564.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kaggle.dropna(inplace=True)


,date,tweet,cleanText,TextBlob Sentiment
0,6/3/2021,@HyperSnoc @Echo31334604 @im_not_kit Racism h...,hypersnoc echo im_not_kit racism,Neutral
1,6/3/2021,@manoutdoors4 @AJ_Lady_Liberty @FBIWFO @TheJus...,manoutdoor aj_lady_liberti fbiwfo thejusticede...,Positive
2,6/3/2021,#TrumpIsNotWell https://t.co/JJl8j4dRK9,trumpisnotwel,Neutral
3,6/3/2021,"@KateWilliamsme @AnnaBethS4 ðŸš¨ ""Racism is ex...",katewilliamsm annabeth ðÿš racism exempt queen...,Neutral
4,6/3/2021,@WGNNews â€˜Critical race theoryâ€™ is nothing...,wgnnew critic race theoryâ noth teach hatr big...,Neutral


In [15]:
kaggle['prediction'] = predict_texts(kaggle['tweet'])

Predicting...: 100%|█████████████████████████████████████████████████████████| 169648/169648 [1:49:05<00:00, 25.92it/s]
C:\Users\MUDIT BAID\AppData\Local\Temp\ipykernel_13200\1984154084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  kaggle['prediction'] = predict_texts(kaggle['tweet'])


In [20]:
kaggle.head()

,date,tweet,cleanText,TextBlob Sentiment,prediction
0,6/3/2021,@HyperSnoc @Echo31334604 @im_not_kit Racism h...,hypersnoc echo im_not_kit racism,Neutral,[0]
1,6/3/2021,@manoutdoors4 @AJ_Lady_Liberty @FBIWFO @TheJus...,manoutdoor aj_lady_liberti fbiwfo thejusticede...,Positive,[0]
2,6/3/2021,#TrumpIsNotWell https://t.co/JJl8j4dRK9,trumpisnotwel,Neutral,[0]
3,6/3/2021,"@KateWilliamsme @AnnaBethS4 ðŸš¨ ""Racism is ex...",katewilliamsm annabeth ðÿš racism exempt queen...,Neutral,[0]
4,6/3/2021,@WGNNews â€˜Critical race theoryâ€™ is nothing...,wgnnew critic race theoryâ noth teach hatr big...,Neutral,[0]


In [35]:
kaggle.to_csv("xlm_kaggle.csv",index=False)

### DATASET 4: TWITTER

In [17]:
from datasets import load_dataset

ds = load_dataset("tweets-hate-speech-detection/tweets_hate_speech_detection")

In [18]:
twitter = ds['train'].to_pandas()
twitter.head()

,label,tweet
0,0,@user when a father is dysfunctional and is so...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [21]:
twitter['prediction'] = predict_texts(twitter['tweet'])

Predicting...: 100%|█████████████████████████████████████████████████████████████| 31962/31962 [19:35<00:00, 27.19it/s]


In [22]:
twitter.head()

,label,tweet,prediction
0,0,@user when a father is dysfunctional and is so...,[0]
1,0,@user @user thanks for #lyft credit i can't us...,[0]
2,0,bihday your majesty,[0]
3,0,#model i love u take with u all the time in ...,[0]
4,0,factsguide: society now #motivation,[0]


In [37]:
twitter.to_csv("xlm_twitter.csv",index=False)